# Практическое задание 1

## Данные о студенте

1. **ФИО**: Селезнев Никита Сергеевич
2. **Факультет**: Физический Факультет
3. **Курс**: 1 магистратура
4. **Группа**: 129м

## Замечания

* Название ноутбука с реализацией должно иметь шаблон "**Prac01_Ivanov.ipynb**" и посылаться на почту mlcoursemm@gmail.com с темой **[CV2021:Prac01]**
* Дедлайн будет оговорен в чате курса
* Соблюдаем кодекс чести (по нулям и списавшему, и давшему списать)
* Можно (и нужно!) применять для реализации только библиотеку **Numpy**
* Ничего, крому Numpy, нельзя использовать для реализации 
* **Keras** используется только для тестирования Вашей реализации
* Если какой-то из классов не проходит приведенные тесты, то соответствующее задание не оценивается
* Возможно использование дополнительных (приватных) тестов
 

In [1]:
# Вам понадобится для реализации
import numpy as np
# Нужно для тестирования
from tensorflow import keras
import keras.layers as layers

* Вспомогательные функции для тестирования

In [2]:
def compare_tensors(x, y, tol=0.001, test_name='Test'):
  assert (x.shape == y.shape), test_name + ' different shapes'
  diff = np.sum((y - x)**2)
  assert (diff < tol), test_name + ' Failed!'
  print (test_name + ' Passed!')
  return

In [3]:
def compare_tensors_array(x, y, tol=0.001, test_name='Test'):
  assert (len(x) == len(y)), test_name + ' different lengths'
  for i in range(len(x)):
    t = test_name + ' subtest ' + str(i)
    compare_tensors(x[i], y[i], tol=tol, test_name=t)
  print (test_name + ' Passed!')
  return

* Шаблон класса любой операции (слоя), которую Вам необходимо будет реализовать

In [4]:
class Layer(object):
    def __init__(self):
        self.name = 'Layer'       
    def forward(self, input_data):
        pass



---



* (1 балл) Реализация "спрямляющего" слоя Flatten

In [5]:
class FlattenLayer(Layer):
    def __init__(self):
      self.name = 'Flatten'
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Преобразуем в двухмерный тензор: при этом по первой размерности НЕ преобразуем
      # Выкладываем данные: сначала по последней размерности, затем по предпоследней и т.д.
      # Нужно заполнить Numpy-тензор out
      batch_size, input_channels_size, height, width = input_data.shape
      out = np.zeros((batch_size, input_channels_size*height*width))
      for i in np.arange(batch_size):
          ind = 0
          for j in np.arange(height):
              for m in np.arange(width):
                  for k in np.arange(input_channels_size):
                      out[i, ind] = input_data[i, k, j , m]
                      ind += 1
      return out

* Функция предварительного тестирования слоя **Flatten**
* Функции с названием "**test_**" не менять
* Вы можете самостоятельно поиграться с параметрами типа B/C/H/W etc

In [6]:
def test_FlattenLayer():
  B = 1
  C = 1
  H = 3
  W = 3
  x = np.random.randn(B, C, H, W)
  y = layers.Flatten(data_format='channels_first')
  y_keras = y(x).numpy()
  y_out = FlattenLayer().forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Flatten 1')
  B = 1
  C = 2
  H = 3
  W = 3
  x = np.random.randn(B, C, H, W)
  y = layers.Flatten(data_format='channels_first')
  y_keras = y(x).numpy()
  y_out = FlattenLayer().forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Flatten 2')
  return

* Запуск теста слоя Flatten
* Нужно, чтобы все тесты были '*Passed!*'

In [7]:
test_FlattenLayer()

Test Flatten 1 Passed!
Test Flatten 2 Passed!




---



* (1 балл) Реализация слоя субдискретизации **Global Average Pooling**

In [8]:
class GAP2DLayer(Layer):
    def __init__(self):
      self.name = 'GAP2D'
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Сворачиваем по двум последним размерностям (то есть на выходе - минус две размерности)
      # Нужно заполнить Numpy-тензор out 
      out = np.mean(input_data, axis=(2, 3))
      return out

In [9]:
def test_GAP2DLayer():
  B = 1
  C = 1
  H = 3
  W = 3
  x = np.random.randn(B, C, H, W)
  y = layers.GlobalAveragePooling2D(data_format='channels_first')
  y_keras = y(x).numpy()
  y_out = GAP2DLayer().forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test GAP2D 1')
  B = 1
  C = 2
  H = 3
  W = 3
  x = np.random.randn(B, C, H, W)
  y = layers.GlobalAveragePooling2D(data_format='channels_first')
  y_keras = y(x).numpy()
  y_out = GAP2DLayer().forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test GAP2D 2')
  return

In [10]:
test_GAP2DLayer()

Test GAP2D 1 Passed!
Test GAP2D 2 Passed!




---



* (2 балла) Реализация слоя субдискретизации **MaxPooling**

In [11]:
class MaxPool2DLayer(Layer):
    def __init__(self, pool_size=2, stride=2):
      self.name = 'MaxPool2D'
      self.pool_size = pool_size
      self.stride = stride
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Нужно заполнить Numpy-тензор out 
      batch_size, input_channels_size, width, height = input_data.shape
      heiht_out = (height - self.pool_size) / self.stride + 1
      width_out = (width - self.pool_size) / self.stride + 1
      out = np.empty((batch_size, input_channels_size, int(heiht_out), int(width_out)))
      for b_i in np.arange(batch_size, dtype=int):
        for ch_i in np.arange(input_channels_size, dtype=int):
          for h_i in np.arange(heiht_out, dtype=int):
              for w_i in np.arange(width_out, dtype=int):
                out[b_i, ch_i, h_i, w_i] = np.max(input_data[b_i, 
                                                  ch_i, 
                                                  h_i*self.stride:(h_i*self.stride + self.pool_size), 
                                                  w_i*self.stride:(w_i*self.stride + self.pool_size)])

      return out

In [12]:
def test_MaxPool2DLayer():
  B = 1
  C = 1
  H = 4
  W = 4
  pool_size = 2
  stride = 2
  x = np.random.randn(B, C, H, W)
  y = layers.MaxPooling2D(pool_size=pool_size, strides=stride, padding="valid", data_format='channels_first')
  y_keras = y(x).numpy()
  y_out = MaxPool2DLayer(pool_size=pool_size, stride=stride).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test MaxPool2D 1')
  B = 2
  C = 2
  H = 3
  W = 3
  pool_size = 2
  stride = 1  
  x = np.random.randn(B, C, H, W)
  y = layers.MaxPooling2D(pool_size=pool_size, strides=stride, padding="valid", data_format='channels_first')
  y_keras = y(x).numpy()
  y_out = MaxPool2DLayer(pool_size=pool_size, stride=stride).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test MaxPool2D 2')
  return

In [13]:
test_MaxPool2DLayer()

Test MaxPool2D 1 Passed!
Test MaxPool2D 2 Passed!




---



* (3 балла) Реализация слоя **активации** (поддерживаются **relu**, **sigmoid**, **softmax**)

In [14]:
class ActivationLayer(Layer):
    def __init__(self, activation='relu'):
      # Активация (поддерживаем 'relu', 'sigmoid', 'softmax')
      self.name = 'Activation'
      self.activation = activation
    def forward(self, input_data):   
      # На входе:
      # четырехмерный тензор вида [batch, input_channels, height, width] для 'relu', 'sigmoid'
      # или двухмерный тензор вида [batch, logits]
      # SoftMax применяется по последней размерности
      # Нужно заполнить Numpy-тензор out 
      out = np.empty(input_data.shape)
      if self.activation == 'relu':
        out = input_data
        out[out < 0] = 0
      elif self.activation == 'sigmoid':
        out = (1 + np.exp(-input_data))**(-1)
      elif self.activation == 'softmax':
        out = np.exp(input_data) / np.exp(input_data).sum(axis=1)[:, np.newaxis]

      return out

In [15]:
def test_ActivationLayer():
  B = 1
  C = 1
  H = 4
  W = 4
  activation = 'relu'
  x = np.random.randn(B, C, H, W)
  y = layers.Activation(activation)
  y_keras = y(x).numpy()
  y_out = ActivationLayer(activation=activation).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Activation 1')
  B = 2
  C = 2
  H = 3
  W = 3
  activation = 'sigmoid'  
  x = np.random.randn(B, C, H, W)
  y = layers.Activation(activation)
  y_keras = y(x).numpy()
  y_out = ActivationLayer(activation=activation).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Activation 2')
  B = 3
  C = 10
  activation = 'softmax'
  x = np.random.randn(B, C)
  y = layers.Activation(activation)
  y_keras = y(x).numpy()
  y_out = ActivationLayer(activation=activation).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Activation 3')  
  return

In [16]:
test_ActivationLayer()

Test Activation 1 Passed!
Test Activation 2 Passed!
Test Activation 3 Passed!




---



* (3 балла) Реализация слоя пакетной нормализации **BatchNorm** (как для режима train, так и для режима test)

In [17]:
# Hint
# Train mode:
# out = (batch - mean(batch)) / sqrt(var(batch) + epsilon) * gamma + beta
# moving_mean = moving_mean * momentum + mean(batch) * (1 - momentum)
# moving_var = moving_var * momentum + var(batch) * (1 - momentum)
# Test mode:
# (batch - moving_mean) / sqrt(moving_var + epsilon) * gamma + beta

class BatchNormLayer(Layer):
    def __init__(self, momentum=0.99, epsilon=0.001, beta_init=None, gamma_init=None,
                 moving_mean_init=None, moving_var_init=None,
                 mode='train', input_channels=2):
      # mode: 'train', 'test'
      # Параметры gamma, beta, mean, var - все имеют размерность по количеству карт input_channels   
      self.name = 'BatchNorm'
      self.momentum = momentum
      self.epsilon = epsilon
      self.beta = beta_init
      self.gamma = gamma_init
      self.moving_mean = moving_mean_init
      self.moving_var = moving_var_init
      self.mode = mode
      self.input_channels = input_channels
    def forward(self, input_data):   
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # 1) Нужно заполнить Numpy-тензор out (той же размерности, что и вход)
      # 2) Нужно обновить moving_mean и moving_var в режиме 'train'
      out = np.empty(input_data.shape)
      if self.mode == 'train':
        self.moving_mean = self.moving_mean * self.momentum + np.mean(input_data,axis=(0,2,3)) * (1 - self.momentum)
        self.moving_var = self.moving_var * self.momentum + np.var(input_data,axis=(0,2,3)) * (1 - self.momentum)
        out = (input_data - np.mean(input_data,axis=(0,2,3)).reshape(1,-1,1,1)) / (np.sqrt(np.var(input_data,axis=(0,2,3)).reshape(1,-1,1,1) + self.epsilon)) * self.gamma.reshape(1,-1,1,1) + self.beta.reshape(1,-1,1,1)
      elif self.mode == 'test':
        out = (input_data - self.moving_mean.reshape(1,-1,1,1)) / (np.sqrt(self.moving_var.reshape(1,-1,1,1) + self.epsilon)) * self.gamma.reshape(1,-1,1,1) + self.beta.reshape(1,-1,1,1)

      return out

In [18]:
def test_BatchNormLayer():
  B = 2
  C = 2
  H = 4
  W = 4
  beta_init = 0 * np.ones(C)
  gamma_init = 1 * np.ones(C)
  moving_mean_init = 0 * np.ones(C)
  moving_var_init= 1 * np.ones(C)
  momentum = 0.99
  epsilon = 0.001
  mode = 'train'
  x = np.random.randn(B, C, H, W)
  y = layers.BatchNormalization(axis=1, momentum=momentum, epsilon=epsilon, trainable=True)
  y_keras = y(x, training=True).numpy()
  y.set_weights([gamma_init, beta_init, moving_mean_init, moving_var_init])
  y_keras = y(x, training=True).numpy()
  y_out_layer = BatchNormLayer(momentum=momentum, epsilon=epsilon, beta_init=beta_init, gamma_init=gamma_init,
                 moving_mean_init=moving_mean_init, moving_var_init=moving_var_init,
                 mode=mode, input_channels=C)
  y_out = y_out_layer.forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test BatchNorm 1')
  compare_tensors_array(y.get_weights(), 
                        [y_out_layer.gamma, y_out_layer.beta, y_out_layer.moving_mean, y_out_layer.moving_var],
                        tol=0.00001, test_name='Test BatchNorm 1.1')
  B = 2 
  C = 2 
  H = 4 
  W = 4 
  beta_init = 1 * np.ones(C)
  gamma_init = 0 * np.ones(C)
  moving_mean = 0 * np.ones(C)
  moving_var = 1 * np.ones(C)
  momentum = 0.99
  epsilon = 0.001
  mode = 'test'
  x = np.random.randn(B, C, H, W)
  y = layers.BatchNormalization(axis=1, momentum=momentum, epsilon=epsilon, trainable=False)
  y_keras = y(x, training=False).numpy()
  y.set_weights([gamma_init, beta_init, moving_mean, moving_var])
  y_keras = y(x, training=False).numpy()
  y_out_layer = BatchNormLayer(momentum=momentum, epsilon=epsilon, beta_init=beta_init, gamma_init=gamma_init,
                 moving_mean_init=moving_mean_init, moving_var_init=moving_var_init,
                 mode=mode, input_channels=C)
  y_out = y_out_layer.forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test BatchNorm 2')  
  compare_tensors_array(y.get_weights(), 
                        [y_out_layer.gamma, y_out_layer.beta, y_out_layer.moving_mean, y_out_layer.moving_var],
                        tol=0.00001, test_name='Test BatchNorm 2.1')
  return

In [19]:
test_BatchNormLayer()

Test BatchNorm 1 Passed!
Test BatchNorm 1.1 subtest 0 Passed!
Test BatchNorm 1.1 subtest 1 Passed!
Test BatchNorm 1.1 subtest 2 Passed!
Test BatchNorm 1.1 subtest 3 Passed!
Test BatchNorm 1.1 Passed!
Test BatchNorm 2 Passed!
Test BatchNorm 2.1 subtest 0 Passed!
Test BatchNorm 2.1 subtest 1 Passed!
Test BatchNorm 2.1 subtest 2 Passed!
Test BatchNorm 2.1 subtest 3 Passed!
Test BatchNorm 2.1 Passed!




---



* (1 балл) Реализация **полносвязного** слоя

In [20]:
class DenseLayer(Layer):
    def __init__(self, input_dim, output_dim, W_init=None, b_init=None):
      self.name = 'Dense'
      self.input_dim = input_dim
      self.output_dim = output_dim
      self.W = W_init
      self.b = b_init
    def forward(self, input_data):
      # На входе - двухмерный тензор вида [batch, input_channels]
      # Работаем по второй размерности, по первой размерности НЕ преобразуем
      # Вначале нужно проверить на согласование размерностей входных данных и ядра!
      # Нужно заполнить Numpy-тензор out
      assert self.W.shape[0] == input_data.shape[1]
      out = np.empty((input_data.shape[0], self.output_dim))
      out = input_data @ self.W + self.b
      return out

In [21]:
def test_DenseLayer():
  B = 1
  C_IN = 10
  C_OUT = 5
  x = np.random.randn(B, C_IN)
  W_init = np.random.randn(C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Dense(C_OUT, use_bias=True)
  y_keras = y(x).numpy()
  y.set_weights([W_init, b_init])
  y_keras = y(x).numpy()
  y_out = DenseLayer(C_IN, C_OUT, W_init=W_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Dense 1')
  B = 2
  C_IN = 5
  C_OUT = 10
  x = np.random.randn(B, C_IN)
  W_init = np.random.randn(C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Dense(C_OUT, use_bias=True, input_shape=(C_IN,))
  y_keras = y(x).numpy()
  y.set_weights([W_init, b_init])
  y_keras = y(x).numpy()
  y_out = DenseLayer(C_IN, C_OUT, W_init=W_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Dense 2')
  return

In [22]:
test_DenseLayer()

Test Dense 1 Passed!
Test Dense 2 Passed!




---



* (2 балла) Реализация **сверточного** слоя

In [23]:
class Conv2DLayer(Layer):
    def __init__(self, kernel_size=3, input_channels=2, output_channels=3, 
                 padding='same', stride=1, K_init=None, b_init=None):
      # padding: 'same' или 'valid'
      # Работаем с квадратными ядрами, поэтому kernel_size - одно число
      # Работаем с единообразным сдвигом, поэтому stride - одно число
      # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
      self.name = 'Conv2D'
      self.kernel_size = kernel_size
      self.input_channels = input_channels
      self.output_channels = output_channels
      self.kernel = K_init
      self.bias = b_init
      self.padding = padding
      self.stride = stride
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Вначале нужно проверить на согласование размерностей входных данных и ядра!
      # Нужно заполнить Numpy-тензор out
      batch_size, input_channels_size, width, height = input_data.shape
      assert self.kernel.shape[2] == input_channels_size, "Dimensions of filter must match dimensions of input image"
      assert (height - self.kernel_size) % self.stride == 0
      assert (width - self.kernel_size) % self.stride == 0
      
      if self.padding == 'same':
        height += 2
        width += 2

      heiht_out = (height - self.kernel_size) / self.stride + 1
      width_out = (width - self.kernel_size) / self.stride + 1
      out = np.empty((batch_size, self.output_channels, int(heiht_out), int(width_out)))

      if self.padding == 'same':
        new_input = np.zeros((batch_size, input_channels_size, height, width))
        new_input[:, :, 1:-1, 1:-1] = input_data
        input_data = np.asarray(new_input)

      for b_i in np.arange(batch_size, dtype=int):
        for ch_i in np.arange(self.output_channels, dtype=int):
          for h_i in np.arange(heiht_out, dtype=int):
              for w_i in np.arange(width_out, dtype=int):
                out[b_i, ch_i, h_i, w_i] = (input_data[
                                                       b_i, 
                                                       :, 
                                                       h_i*self.stride:(h_i*self.stride + self.kernel_size), 
                                                       w_i*self.stride:(w_i*self.stride + self.kernel_size)
                                                       ] \
                                            * np.moveaxis(self.kernel[:, :, :, ch_i], -1, 0)
                                                       ).sum()

      out = out + self.bias.reshape(1, -1, 1, 1)

      return out

In [24]:
def test_Conv2DLayer():
  B = 1
  C_IN = 1
  C_OUT = 1
  H = 10
  W = 10
  K = 3
  S = 1
  padding = 'same'
  x = np.random.randn(B, C_IN, H, W)
  K_init = np.random.randn(K, K, C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Conv2D(C_OUT, K, strides=S, padding=padding, data_format='channels_first',
    dilation_rate=1, groups=1, activation=None, use_bias=True)
  y_keras = y(x).numpy()
  y.set_weights([K_init, b_init])
  y_keras = y(x).numpy()
  y_out = Conv2DLayer(kernel_size=K, input_channels=C_IN, output_channels=C_OUT, 
                 padding=padding, stride=S, K_init=K_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Conv2D 1')
  B = 2
  C_IN = 3
  C_OUT = 5
  H = 9
  W = 9
  K = 3
  S = 2
  padding = 'valid'
  x = np.random.randn(B, C_IN, H, W)
  K_init = np.random.randn(K, K, C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Conv2D(C_OUT, K, strides=S, padding=padding, data_format='channels_first',
    dilation_rate=1, groups=1, activation=None, use_bias=True, input_shape=(C_IN, H, W))
  y_keras = y(x).numpy()
  y.set_weights([K_init, b_init])
  y_keras = y(x).numpy()
  y_out = Conv2DLayer(kernel_size=K, input_channels=C_IN, output_channels=C_OUT, 
                 padding=padding, stride=S, K_init=K_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Conv2D 1')
  return

In [25]:
test_Conv2DLayer()

Test Conv2D 1 Passed!
Test Conv2D 1 Passed!




---



* (2 балла) Реализация **транспонированного сверточного** слоя

In [26]:
class Conv2DTrLayer(Layer):
    def __init__(self, kernel_size=3, input_channels=2, output_channels=3, 
                 padding=0, stride=1, K_init=None, b_init=None):      
      # padding: число (сколько отрезать от модифицированной входной карты)
      # Работаем с квадратными ядрами, поэтому kernel_size - одно число
      # stride - одно число (коэффициент расширения)
      # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
      self.name = 'Conv2DTr'
      self.kernel_size = kernel_size
      self.input_channels = input_channels
      self.output_channels = output_channels
      self.kernel = K_init
      self.bias = b_init
      self.padding = padding
      self.stride = stride
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Вначале нужно проверить на согласование размерностей входных данных и ядра!
      # Нужно заполнить Numpy-тензор out

      out = np.empty([])
      return out

In [27]:
def adjust_kernel(K):
  K_new = K.copy()[::-1, ::-1, :, :]
  K_new = np.transpose(K_new, (0, 1, 3, 2))
  return K_new

def test_Conv2DTrLayer():
  B = 1
  C_IN = 1
  C_OUT = 1
  H = 3
  W = 3
  K = 3
  S = 2
  padding = 0
  x = np.random.randn(B, C_IN, H, W)
  K_init = np.random.randn(K, K, C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Conv2DTranspose(C_OUT, K, strides=S, padding="valid", output_padding=None, 
                    data_format='channels_first', dilation_rate=1, groups=1, 
                    activation=None, use_bias=True)
  y_keras = y(x).numpy()
  y.set_weights([adjust_kernel(K_init), b_init])
  y_keras = y(x).numpy()
  y_out = Conv2DTrLayer(kernel_size=K, input_channels=C_IN, output_channels=C_OUT, 
                 padding=padding, stride=S, K_init=K_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Conv2DTr 1')
  B = 4
  C_IN = 2
  C_OUT = 3
  H = 3
  W = 3
  K = 3
  S = 2
  padding = 0
  x = np.random.randn(B, C_IN, H, W)
  K_init = np.random.randn(K, K, C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Conv2DTranspose(C_OUT, K, strides=S, padding="valid", output_padding=None, 
                    data_format='channels_first', dilation_rate=1, groups=1, 
                    activation=None, use_bias=True)
  y_keras = y(x).numpy()
  y.set_weights([adjust_kernel(K_init), b_init])
  y_keras = y(x).numpy()
  y_out = Conv2DTrLayer(kernel_size=K, input_channels=C_IN, output_channels=C_OUT, 
                 padding=padding, stride=S, K_init=K_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Conv2DTr 2')
  return

In [28]:
test_Conv2DTrLayer()

AssertionError: ignored